In [5]:
import numpy as np
import pandas as pd
from scipy import sparse
from scipy import cluster
from scipy import spatial
from matplotlib import pyplot as plt
import seaborn
import copy
%matplotlib inline

# Clustering by tags

### Data Cleaning

In [6]:
big_set = pd.read_csv('./data/artists.csv', low_memory=False)

In [7]:
artist_tags=big_set[['artist_mb', 'tags_mb']]

In [8]:
artist_tags = artist_tags.dropna()
artist_tags

,artist_mb,tags_mb
0,Coldplay,rock; pop; alternative rock; british; uk; brit...
1,Radiohead,rock; electronic; alternative rock; british; g...
2,Red Hot Chili Peppers,rock; alternative rock; 80s; 90s; rap; metal; ...
3,Rihanna,pop; dance; hip hop; reggae; contemporary r b;...
4,Eminem,turkish; rap; american; hip-hop; hip hop; hiph...
...,...,...
1466059,Kazushige Kinoshita,japanese; violinist; japan; chamber music; fre...
1466061,水越恵子,likedis auto
1466063,大槻ケンヂ,rock
1466069,孫耀威,chinese


In [9]:
artist_tags['tags_mb'] = artist_tags['tags_mb'].apply(lambda x: np.array([tag.strip() for tag in x.split(';')]))
artist_tags

,artist_mb,tags_mb
0,Coldplay,"[rock, pop, alternative rock, british, uk, bri..."
1,Radiohead,"[rock, electronic, alternative rock, british, ..."
2,Red Hot Chili Peppers,"[rock, alternative rock, 80s, 90s, rap, metal,..."
3,Rihanna,"[pop, dance, hip hop, reggae, contemporary r b..."
4,Eminem,"[turkish, rap, american, hip-hop, hip hop, hip..."
...,...,...
1466059,Kazushige Kinoshita,"[japanese, violinist, japan, chamber music, fr..."
1466061,水越恵子,[likedis auto]
1466063,大槻ケンヂ,[rock]
1466069,孫耀威,[chinese]


In [10]:
artist_tags_len = artist_tags
artist_tags_len['tags_len'] = artist_tags_len['tags_mb'].apply(lambda x: len(x))

In [11]:
np_artists = artist_tags_len['tags_mb'].to_numpy()

In [12]:
np_artists

array([array(['rock', 'pop', 'alternative rock', 'british', 'uk', 'britannique',
              'britpop', 'pop rock', 'piano pop', 'piano rock', 'english',
              'parlophone', 'rock and indie', 'ambient pop', 'pop/rock',
              'chapel', 'post-britpop'], dtype='<U16')                          ,
       array(['rock', 'electronic', 'alternative rock', 'british', 'grunge',
              'uk', 'britannique', 'britpop', 'art rock', 'experimental rock',
              'english', 'chamber pop', 'parlophone', 'england', 'melancholic',
              'oxford', 'bootleg', 'rock and indie', 'c’était mieux avant',
              'art pop', 'nude', 'sacred cows'], dtype='<U19')                 ,
       array(['rock', 'alternative rock', '80s', '90s', 'rap', 'metal',
              'american', 'crossover', 'usa', 'funk', 'funk rock', 'alternative',
              'pop rock', 'funk metal', 'rap rock', '00s', 'dvd',
              'pop and chart', '10s', 'funk rock tributo'], dtype='<U17')   

In [13]:
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in np_artists:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))
    

tag_matrix = sparse.csr_matrix((data, indices, indptr), dtype=int)
tmr = tag_matrix.toarray()

In [14]:
tag_matrix

<119943x36091 sparse matrix of type '<class 'numpy.int64'>'
	with 305601 stored elements in Compressed Sparse Row format>

In [15]:
from scipy.sparse import * 
import sklearn.cluster as cluster 

In [16]:
labeler = cluster.KMeans() 

In [17]:
labeler.fit(tag_matrix)

KMeans()

In [18]:
labeler.labels_

array([3, 3, 6, ..., 6, 7, 7], dtype=int32)